# DAMR
Hi, here is the code of DAMR(Dynamic Ajacency Matrix Representation) 
for data imputation in COVID-19 dataset.

# 1. Import library

In [5]:
!pip install keras

  Using cached keras-2.9.0-py2.py3-none-any.whl (1.6 MB)


In [9]:
!pip install tensorflow
!pip install spectral

In [10]:
!pip install spektral


In [16]:
#1.Import library
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import networkx as nx
from networkx.algorithms import bipartite
import copy
from sklearn.preprocessing import StandardScaler
import scipy.special as special
import scipy.sparse as sp
import pickle
import os
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

import tensorflow as tf
#import tensorflow_addons as tfa
import keras
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense, GRU, Input
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dropout, Dense, LeakyReLU, GRU, Concatenate, Reshape, Softmax, Attention
from tensorflow.keras import activations
from tensorflow.keras.models import Model
#from keras.layers import LeakyReLU
from spektral.layers import  GCSConv, DiffusionConv, GATConv, ARMAConv, GCNConv
from tensorflow.keras.regularizers import l2
#from my own library import Generate_missing and other models
from model import * #generate_miss(data,perc),output:data after generate missing.
import FancyImpute
from FancyImpute import *
import torch
#Bidirectional LSTM
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Bidirectional
from BiLSTM import BiLSTM_3D

In [14]:
!pip install FancyImpute

  Created wheel for FancyImpute: filename=fancyimpute-0.7.0-py3-none-any.whl size=29899 sha256=a0e0c75b1c755a4697f8387fdd0ca86eba9035aa985fffad6ac3998ca3fa81e1
  Stored in directory: c:\users\77154\appdata\local\pip\cache\wheels\f9\fc\6a\b0406b906bce293abe23c3b6da5a72637d2d04146ef1125a0b
  Created wheel for knnimpute: filename=knnimpute-0.1.0-py3-none-any.whl size=11353 sha256=803a018f1e63a68d500afcc883f651769724bd168d913ce2ffa1f0fb96dd68ce
  Stored in directory: c:\users\77154\appdata\local\pip\cache\wheels\88\c4\be\e232c750d9bc360abf9a5e2cafe0d3e08e3605d2801bb11684
Successfully built FancyImpute knnimpute


# 2. Data Pre-processing of COVID dataset

In [15]:
!pip install autorank

In [17]:
#Define hyperparameter
Miss_perc=0.7
Split_perc1=0.8
Split_perc2=0.1


In [18]:
#Generate Missing 3D.
path='Data/COVID/raw'
COVID_features=Csv2Tensor(path)
COVID_features=COVID_features[:503,:,:]
COVID_features=MinMax3D(COVID_features)#(503,3,7)
COVID_features_train=COVID_features[:int(Split_perc1*COVID_features.shape[0]),:,:]
print(COVID_features_train.shape)#(402, 3, 7)
COVID_features_val=COVID_features[int(Split_perc1*COVID_features.shape[0]):int(Split_perc1*COVID_features.shape[0])+int(Split_perc2*COVID_features.shape[0]),:,:]
print(COVID_features_val.shape)#(50, 3, 7)
COVID_features_test=COVID_features[int(Split_perc1*COVID_features.shape[0])+int(Split_perc2*COVID_features.shape[0]):,:,:]
print(COVID_features_test.shape)#(51, 3, 7)

#The mask and missing data has been created with a given miss percentage.
#It will be used for baseline and for DAMR
COVID_features_train_miss,COVID_features_train_mask=generate_miss_3D(COVID_features_train,Miss_perc)
COVID_features_val_miss,COVID_features_val_mask=generate_miss_3D(COVID_features_val,Miss_perc)
COVID_features_test_miss,COVID_features_test_mask=generate_miss_3D(COVID_features_test,Miss_perc)
COVID_features_miss,COVID_features_mask=generate_miss_3D(COVID_features,Miss_perc)


()
()
()
()
()
()
()
(402, 3, 7)
(50, 3, 7)
(51, 3, 7)


In [19]:
#BiLSTM_finish
COVID_features_miss,COVID_features_mask=generate_miss_3D(COVID_features,0.1)
BiLSTM_rmse,BiLSTM_mae,BiLSTM_mape,BiLSTM_mre,BiLSTM_err=BiLSTM_3D(COVID_features_miss,COVID_features,COVID_features_mask,0)
print("BiLSTM_rmse,",BiLSTM_rmse)
print("BiLSTM_mae",BiLSTM_mae)
print("BiLSTM_mape",BiLSTM_mape)
print("BiLSTM_mre",BiLSTM_mre)
print("BiLSTM_err",BiLSTM_err)

(402, 1, 3) (402, 3) (100, 1, 3) (100, 3)
Epoch 1/80
1/1 - 3s - loss: 0.5406 - val_loss: 0.7403 - 3s/epoch - 3s/step
Epoch 2/80
1/1 - 0s - loss: 0.5366 - val_loss: 0.7561 - 22ms/epoch - 22ms/step
Epoch 3/80
1/1 - 0s - loss: 0.5326 - val_loss: 0.7715 - 19ms/epoch - 19ms/step
Epoch 4/80
1/1 - 0s - loss: 0.5288 - val_loss: 0.7866 - 16ms/epoch - 16ms/step
Epoch 5/80
1/1 - 0s - loss: 0.5251 - val_loss: 0.8014 - 16ms/epoch - 16ms/step
Epoch 6/80
1/1 - 0s - loss: 0.5215 - val_loss: 0.8159 - 17ms/epoch - 17ms/step
Epoch 7/80
1/1 - 0s - loss: 0.5180 - val_loss: 0.8302 - 18ms/epoch - 18ms/step
Epoch 8/80
1/1 - 0s - loss: 0.5146 - val_loss: 0.8442 - 20ms/epoch - 20ms/step
Epoch 9/80
1/1 - 0s - loss: 0.5113 - val_loss: 0.8580 - 17ms/epoch - 17ms/step
Epoch 10/80
1/1 - 0s - loss: 0.5081 - val_loss: 0.8716 - 19ms/epoch - 19ms/step
Epoch 11/80
1/1 - 0s - loss: 0.5049 - val_loss: 0.8849 - 17ms/epoch - 17ms/step
Epoch 12/80
1/1 - 0s - loss: 0.5019 - val_loss: 0.8980 - 18ms/epoch - 18ms/step
Epoch 13/80

In [20]:
#Run baseline(KNN,MICE,MF,MEAN,SLIDING)#Return:RMSE,MAE,MAPE,MRE
#For the model of FancyImpute,we just need to test.
KNN_RMSE,KNN_MAE,KNN_MAPE,KNN_MRE=KNN_impute_3D(COVID_features_test_miss,COVID_features_test,COVID_features_test_mask)
MICE_RMSE,MICE_MAE,MICE_MAPE,MICE_MRE=MICE_impute_3D(COVID_features_test_miss,COVID_features_test,COVID_features_test_mask)
MF_RMSE,MF_MAE,MF_MAPE,MF_MRE=MF_impute_3D(COVID_features_test_miss,COVID_features_test,COVID_features_test_mask)
MEAN_RMSE,MEAN_MAE,MEAN_MAPE,MEAN_MRE=MEAN_impute_3D(COVID_features_test_miss,COVID_features_test,COVID_features_test_mask)
Sliding_RMSE,Sliding_MAE,Sliding_MAPE,Sliding_MRE,Sliding_out=Sliding_impute_3D(COVID_features_test_miss,COVID_features_test,COVID_features_test_mask,6)
print("KNN_RMSE",KNN_RMSE,"KNN_MAE",KNN_MAE,"KNN_MAPE",KNN_MAPE,"KNN_MRE",KNN_MRE)
print("MICE_RMSE",MICE_RMSE,"MICE_MAE",MICE_MAE,"MICE_MAPE",MICE_MAPE,"MICE_MRE",MICE_MRE)
print("MF_RMSE",MF_RMSE,"MF_MAE",MF_MAE,"MF_MAPE",MF_MAPE,"MF_MRE",MF_MRE)
print("MEAN_RMSE",MEAN_RMSE,"MEAN_MAE",MEAN_MAE,"MEAN_MAPE",MEAN_MAPE,"MEAN_MRE",MEAN_MRE)
print("Sliding_RMSE",Sliding_RMSE,"Sliding_MAE",Sliding_MAE,"Sliding_MAPE",Sliding_MAPE,"Sliding_MRE",Sliding_MRE)

Imputing row 1/51 with 3 missing, elapsed time: 0.000
[KNN] Warning: 45/153 still missing after imputation, replacing with 0
Imputing row 1/51 with 3 missing, elapsed time: 0.000
[KNN] Warning: 42/153 still missing after imputation, replacing with 0
Imputing row 1/51 with 3 missing, elapsed time: 0.000
[KNN] Warning: 48/153 still missing after imputation, replacing with 0
Imputing row 1/51 with 3 missing, elapsed time: 0.000
[KNN] Warning: 72/153 still missing after imputation, replacing with 0
Imputing row 1/51 with 3 missing, elapsed time: 0.000
[KNN] Warning: 51/153 still missing after imputation, replacing with 0
Imputing row 1/51 with 3 missing, elapsed time: 0.000
[KNN] Warning: 69/153 still missing after imputation, replacing with 0
Imputing row 1/51 with 3 missing, elapsed time: 0.000
[KNN] Warning: 51/153 still missing after imputation, replacing with 0
                                     CVXPY                                     
                                     v1.2.1  

D:\Software\Installation\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


     pcost       dcost       gap    pres   dres   k/t
 0:  0.0000e+00 -1.5300e-02  2e+03  8e+00  4e+00  1e+00
 1:  1.2096e+01  1.3450e+01  4e+02  2e+00  1e+00  2e+00
 2:  8.9842e+00  9.7253e+00  1e+02  1e+00  5e-01  9e-01
 3:  9.3313e+00  9.4051e+00  1e+01  8e-02  4e-02  8e-02
 4:  9.3291e+00  9.3760e+00  7e+00  6e-02  3e-02  5e-02
 5:  9.3284e+00  9.3349e+00  1e+00  1e-02  5e-03  8e-03
 6:  9.3318e+00  9.3323e+00  2e-01  2e-03  7e-04  7e-04
 7:  9.3367e+00  9.3367e+00  2e-02  3e-04  1e-04  3e-05
 8:  9.3392e+00  9.3392e+00  2e-03  8e-05  4e-05  1e-05
 9:  9.3397e+00  9.3397e+00  4e-04  2e-05  8e-06  3e-06
10:  9.3398e+00  9.3398e+00  4e-05  2e-06  9e-07  2e-07
11:  9.3399e+00  9.3399e+00  4e-06  2e-07  9e-08  1e-08
12:  9.3399e+00  9.3399e+00  2e-07  8e-09  4e-09  4e-10
Optimal solution found.
-------------------------------------------------------------------------------
                                    Summary                                    
----------------------------------

-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jun 05 12:49:45 AM: Compiling problem (target solver=CVXOPT).
(CVXPY) Jun 05 12:49:45 AM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> CVXOPT
(CVXPY) Jun 05 12:49:45 AM: Applying reduction Dcp2Cone
(CVXPY) Jun 05 12:49:45 AM: Applying reduction CvxAttr2Constr
(CVXPY) Jun 05 12:49:45 AM: Applying reduction ConeMatrixStuffing
(CVXPY) Jun 05 12:49:45 AM: Applying reduction CVXOPT
(CVXPY) Jun 05 12:49:45 AM: Finished problem compilation (took 1.397e-02 seconds).
-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
(CVXPY) Jun 05 12:49:45 A

(CVXPY) Jun 05 12:49:59 AM: Optimal value: 8.732e+00
(CVXPY) Jun 05 12:49:59 AM: Compilation took 1.200e-02 seconds
(CVXPY) Jun 05 12:49:59 AM: Solver (including time spent in interface) took 4.928e+00 seconds
                                     CVXPY                                     
                                     v1.2.1                                    
(CVXPY) Jun 05 12:49:59 AM: Your problem has 153 variables, 1 constraints, and 0 parameters.
(CVXPY) Jun 05 12:49:59 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jun 05 12:49:59 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jun 05 12:49:59 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
---------------------------------

# Run Baseline

In [21]:
##Lags for variable 1 ,modify input variable here!
COVID_X1_all=COVID_features[:,0,:]
#Transform the shape into Bidirectional problems
COVID_X1_all=series_to_supervised(COVID_X1_all,15,16)
COVID_Y1_all=COVID_X1_all.iloc[:,15*7:15*7+7]
COVID_X1_all_1=COVID_X1_all.iloc[:,:15*7]
COVID_X1_all_2=COVID_X1_all.iloc[:,15*7+7:]
COVID_X1_all=np.hstack((COVID_X1_all_1,COVID_X1_all_2))
COVID_X1_all=np.array(COVID_X1_all)
COVID_X1_all=np.reshape(COVID_X1_all,(len(COVID_X1_all),COVID_features.shape[2],30))
#Input 1->Groundtruth
COVID_features=COVID_features[15:-15,:,:]#(473, 3, 7)
print(COVID_features.shape)
#Input2->Groundtruth
print(COVID_X1_all.shape)#(473,7,30)
COVID_Y1_all=np.array(COVID_Y1_all)
COVID_Y1_all=np.reshape(COVID_Y1_all,(len(COVID_Y1_all),7,1))
#Input3->Groundtruth
Adj_all = np.arange(473*7*7).reshape(473,7,7)#(473,7,7)
#Output1->Groundtruth
print(COVID_Y1_all.shape)#(473, 7, 1)

(473, 3, 7)
(473, 7, 30)
(473, 7, 1)


In [7]:
##Input and output in DAMR!
# print(COVID_features_aftslid.shape)#473*3*7
# print(COVID_X1_all_aftslid.shape)#473*7*30
# print(COVID_Y1_all.shape)#(473, 7, 1)
# print(Adj_dist.shape)#7*7

#The mask and missing data has been created with a given miss percentage.
#It will be used for baseline and for DAMR
#Generate Missing 3D.
COVID_features=Csv2Tensor(path)
COVID_features=COVID_features[:503,:,:]
COVID_features=MinMax3D(COVID_features)#(503,3,7)->Groundtruth
COVID_features_train=COVID_features[:int(Split_perc1*COVID_features.shape[0]),:,:]
print(COVID_features_train.shape)#(402, 3, 7)
COVID_features_val=COVID_features[int(Split_perc1*COVID_features.shape[0]):int(Split_perc1*COVID_features.shape[0])+int(Split_perc2*COVID_features.shape[0]),:,:]
print(COVID_features_val.shape)#(50, 3, 7)
COVID_features_test=COVID_features[int(Split_perc1*COVID_features.shape[0])+int(Split_perc2*COVID_features.shape[0]):,:,:]
print(COVID_features_test.shape)#(51, 3, 7)

#
COVID_features_train_miss,COVID_features_train_mask=generate_miss_3D(COVID_features_train,Miss_perc)#(402, 3, 7)
COVID_features_val_miss,COVID_features_val_mask=generate_miss_3D(COVID_features_val,Miss_perc)#(50, 3, 7)
COVID_features_test_miss,COVID_features_test_mask=generate_miss_3D(COVID_features_test,Miss_perc)#(51, 3, 7)
COVID_features_miss,COVID_features_mask=generate_miss_3D(COVID_features,Miss_perc)

Sliding_RMSE,Sliding_MAE,Sliding_MAPE,Sliding_MRE,Sliding_out=Sliding_impute_3D(COVID_features_miss,COVID_features,COVID_features_mask,6)#(503,3,7)




()
()
()
()
()
()
()
(402, 3, 7)
(50, 3, 7)
(51, 3, 7)


In [8]:
#Create lags for the first variable.
#Extract the first variable
lags=30
var=0
Sliding_out_var1=Sliding_out[:,var,:]
Sliding_out_var1_sup=series_to_supervised(Sliding_out_var1,lags,1)
Sliding_out_var1_sup=Sliding_out_var1_sup.iloc[:,:lags*Sliding_out.shape[2]]
print(Sliding_out_var1_sup)
#Input1: 473*7*30
COVID_X1_all_aftslid=np.array(Sliding_out_var1_sup).reshape(len(Sliding_out_var1_sup),COVID_features.shape[2],lags)
#Input2: 473*3*7
COVID_features_aftslid=Sliding_out[30:,:,:]
#Input3: Adj: 473*7*7(Initialization)
Adj_all = np.arange(len(COVID_features_aftslid)*COVID_features_aftslid.shape[2]*COVID_features_aftslid.shape[2]).reshape(len(COVID_features_aftslid),COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2])#(473,7,7)
#Output:Groundtruth of the first variable->
COVID_Y1_all=COVID_features[30:,:,:]
COVID_Y1_all=COVID_Y1_all[:,var,:]
COVID_Y1_all=COVID_Y1_all.reshape(COVID_Y1_all.shape[0],COVID_Y1_all.shape[1],1)
#
print(COVID_X1_all_aftslid.shape,COVID_features_aftslid.shape,Adj_all.shape,COVID_Y1_all.shape)

     var1(t-30)  var2(t-30)  var3(t-30)  var4(t-30)  var5(t-30)  var6(t-30)  \
30    -1.000000   -1.000000   -1.000000   -1.000000   -1.000000   -0.999887   
31    -0.999921   -1.000000   -0.999681   -0.999904   -0.999977   -0.999801   
32    -0.999847   -1.000000   -0.999681   -0.999904   -0.999944   -1.000000   
33    -0.999848   -1.000000   -0.999681   -0.999904   -0.999944   -0.999894   
34    -0.999847   -1.000000   -0.999362   -0.999904   -0.999855   -0.999767   
..          ...         ...         ...         ...         ...         ...   
498    0.928060    0.846354    0.892140    0.765349    0.808057    0.657038   
499    0.927294    0.852555    0.903089    0.777349    0.866284    0.540383   
500    0.928055    0.860718    0.912583    0.769372    0.819790    0.577787   
501    0.928807    0.865893    0.920835    0.783582    0.824582    0.612895   
502    0.929354    0.863381    0.924703    0.784850    0.829209    0.625041   

     var7(t-30)  var1(t-29)  var2(t-29)  var3(t-29)

In [9]:
type(Sliding_out_var1_sup)

pandas.core.frame.DataFrame

# 3.Dynamic Adjacency matrix layer

In [10]:
#3.1 Distance-based adjacency matrix 
#Country: BZ CL ID IR MX PR SA.
COVID_lat=pd.DataFrame([-14.235,-35.6751,20.5937,32.4279,23.6345,-9.19,30.5595])
COVID_long=pd.DataFrame([-51.9253,-71.543,78.9629,53.688,-102.5528,-75.0152,22.9375])
Adj_dist=adj_dist(COVID_lat,COVID_long)#7*7

In [11]:
data1=pd.DataFrame([[-51.9253,-71.543,78.9629],[53.688,-102.5528,-75.0152]])
data2=pd.DataFrame([[-31.9253,-41.543,38.9629],[43.688,62.5528,65.0152]])
Global_MeanMI(data1,data2,2)
data1=pd.DataFrame(data1)
data2=pd.DataFrame(data2)
data1=np.array(data1).reshape(-1)
data2=np.array(data2).reshape(-1)

In [12]:
#3.2 Mean MI-based adjacency matrix
print(COVID_features_aftslid.shape)
MeanMI=[]
for i in range(COVID_features_aftslid.shape[2]):
    for j in range(COVID_features_aftslid.shape[2]):
        MeanMI.append(Global_MeanMI(COVID_features_aftslid[:,:,i],
                                    COVID_features_aftslid[:,:,j],2))
print(np.array(MeanMI).shape)
MeanMI=np.array(MeanMI).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2])
print(MeanMI.shape)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled1 = scaler.fit_transform(MeanMI)
MeanMI=scaled1#shape:7*7
print(MeanMI)

(473, 3, 7)
(49,)
(7, 7)
[[1.         0.07716456 0.68564709 0.69383481 0.5370537  0.83884206
  0.25584898]
 [0.         1.         0.         0.         0.         0.
  0.        ]
 [0.66732929 0.18265125 1.         0.49899386 0.64287186 0.61331621
  0.42425451]
 [0.69864404 0.         0.48417807 1.         0.3671463  0.80207329
  0.29476106]
 [0.53379177 0.23737679 0.6322874  0.4107627  1.         0.47187885
  0.21619425]
 [0.84687459 0.04413379 0.62510737 0.79660119 0.45633642 1.
  0.23329727]
 [0.31485467 0.29141682 0.4262656  0.36273766 0.23348526 0.30773986
  1.        ]]


In [13]:
#3.3 CONCAT MI-based adjacency matrix
print(COVID_features_aftslid.shape)
ConcatMI=[]
for i in range(COVID_features_aftslid.shape[2]):
    for j in range(COVID_features_aftslid.shape[2]):
        ConcatMI.append(Global_MeanMI(COVID_features_aftslid[:,:,i],
                                    COVID_features_aftslid[:,:,j],2))
ConcatMI=np.array(ConcatMI).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2])
scaler = MinMaxScaler(feature_range=(0, 1))
scaled1 = scaler.fit_transform(ConcatMI)
ConcatMI=scaled1#shape:7*7
print(ConcatMI)

(473, 3, 7)
[[1.         0.07716456 0.68564709 0.69383481 0.5370537  0.83884206
  0.25584898]
 [0.         1.         0.         0.         0.         0.
  0.        ]
 [0.66732929 0.18265125 1.         0.49899386 0.64287186 0.61331621
  0.42425451]
 [0.69864404 0.         0.48417807 1.         0.3671463  0.80207329
  0.29476106]
 [0.53379177 0.23737679 0.6322874  0.4107627  1.         0.47187885
  0.21619425]
 [0.84687459 0.04413379 0.62510737 0.79660119 0.45633642 1.
  0.23329727]
 [0.31485467 0.29141682 0.4262656  0.36273766 0.23348526 0.30773986
  1.        ]]


In [14]:
#3.4 Single Feature MI-based adjacency matrix
COVID_features_aftslid.shape
FeatureMI=[]
for i in range(COVID_features_aftslid.shape[2]):
    for j in range(COVID_features_aftslid.shape[2]):
        FeatureMI.append(Global_EachMI(COVID_features_aftslid[:,:,i],
                                    COVID_features_aftslid[:,:,j],2))
#For the first feature,if choose the second feature, modify here!!
FeatureMI=np.array(FeatureMI)[:,2]
FeatureMI=np.array(FeatureMI).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2])
scaler = MinMaxScaler(feature_range=(0, 1))
scaled1 = scaler.fit_transform(FeatureMI)
FeatureMI=scaled1#shape:7*7
#MeanMI,ConcatMI,FeatureMI

In [15]:
#3.5 Mean MI-based dynamic adjacency matrix
print(COVID_features_aftslid.shape)
#Index all of the rows and columns
DyMeanMI=[]
for k in range(COVID_features_aftslid.shape[0]):
    for i in range(COVID_features_aftslid.shape[2]):
        for j in range(COVID_features_aftslid.shape[2]):
            DyMeanMI.append(Global_MeanMI(COVID_features_aftslid[:k,:,i],
                                    COVID_features_aftslid[:k,:,j],2))
DyMeanMI=np.array(DyMeanMI).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[0])
DyMeanMI=DyMeanMI.transpose(2, 0, 1)
from fancyimpute import SimpleFill, NuclearNormMinimization, SoftImpute, BiScaler
for i in range(DyMeanMI.shape[0]):
    MEAN_imputer = SimpleFill()
    DyMeanMI[i,:,:]=MEAN_imputer.fit_transform(DyMeanMI[i,:,:])
print(DyMeanMI)#Here DyMeanMI is done

(473, 3, 7)


/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))


[[[0.55191962 0.86693206 0.52608262 ... 0.41248463 0.34804301 0.53409661]
  [0.55530318 0.38395281 0.22546964 ... 0.33223676 0.33156705 0.90395516]
  [0.29076203 0.60034246 0.33424337 ... 0.48181446 0.50248129 0.69809963]
  ...
  [0.62737299 0.33615017 0.65943872 ... 0.29130526 0.70338862 0.5157269 ]
  [0.37960582 0.7652817  0.99843895 ... 0.63402173 0.57477868 0.80817513]
  [0.87710857 0.60521372 0.18074768 ... 0.64380001 0.48289189 0.58396495]]

 [[0.47793464 0.17755509 0.94906555 ... 0.52221856 0.26343453 0.3119398 ]
  [0.45924516 0.36561905 0.59832988 ... 0.39096349 0.65427623 0.59169737]
  [0.64133567 0.89315441 0.48809894 ... 0.7648398  0.68220701 0.77535175]
  ...
  [0.31045698 0.42997357 0.57477863 ... 0.71257367 0.55869885 0.8166429 ]
  [0.51170629 0.75101508 0.72505545 ... 0.94752128 0.84383326 0.7043459 ]
  [0.62811498 0.84731386 0.56494582 ... 0.702848   0.81969185 0.68847023]]

 [[0.5135696  0.21472315 0.62949329 ... 0.82618946 0.41955481 0.26725408]
  [0.38554379 0.299911

/opt/anaconda3/lib/python3.8/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/opt/anaconda3/lib/python3.8/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/opt/anaconda3/lib/python3.8/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/opt/anaconda3/lib/python3.8/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/opt/anaconda3/lib/python3.8/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/opt/anaconda3/lib/python3.8/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not mis

In [16]:
#3.6 Concat MI-based dynamic adjacency matrix
#Index all of the rows and columns
DyConcatMI=[]
for k in range(COVID_features_aftslid.shape[0]):
    for i in range(COVID_features_aftslid.shape[2]):
        for j in range(COVID_features_aftslid.shape[2]):
            DyConcatMI.append(Global_MeanMI(COVID_features_aftslid[:k,:,i],
                                    COVID_features_aftslid[:k,:,j],2))
DyConcatMI=np.array(DyConcatMI).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[0])
DyConcatMI=DyConcatMI.transpose(2, 0, 1)
from fancyimpute import SimpleFill, NuclearNormMinimization, SoftImpute, BiScaler
for i in range(DyConcatMI.shape[0]):
    MEAN_imputer = SimpleFill()
    DyConcatMI[i,:,:]=MEAN_imputer.fit_transform(DyConcatMI[i,:,:])
print(DyConcatMI)#Here DyConcatMI is done

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/

/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in 

/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in 

[[[0.55191962 0.86693206 0.52608262 ... 0.41248463 0.34804301 0.53409661]
  [0.55530318 0.38395281 0.22546964 ... 0.33223676 0.33156705 0.90395516]
  [0.29076203 0.60034246 0.33424337 ... 0.48181446 0.50248129 0.69809963]
  ...
  [0.62737299 0.33615017 0.65943872 ... 0.29130526 0.70338862 0.5157269 ]
  [0.37960582 0.7652817  0.99843895 ... 0.63402173 0.57477868 0.80817513]
  [0.87710857 0.60521372 0.18074768 ... 0.64380001 0.48289189 0.58396495]]

 [[0.47793464 0.17755509 0.94906555 ... 0.52221856 0.26343453 0.3119398 ]
  [0.45924516 0.36561905 0.59832988 ... 0.39096349 0.65427623 0.59169737]
  [0.64133567 0.89315441 0.48809894 ... 0.7648398  0.68220701 0.77535175]
  ...
  [0.31045698 0.42997357 0.57477863 ... 0.71257367 0.55869885 0.8166429 ]
  [0.51170629 0.75101508 0.72505545 ... 0.94752128 0.84383326 0.7043459 ]
  [0.62811498 0.84731386 0.56494582 ... 0.702848   0.81969185 0.68847023]]

 [[0.5135696  0.21472315 0.62949329 ... 0.82618946 0.41955481 0.26725408]
  [0.38554379 0.299911

/opt/anaconda3/lib/python3.8/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/opt/anaconda3/lib/python3.8/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/opt/anaconda3/lib/python3.8/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/opt/anaconda3/lib/python3.8/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/opt/anaconda3/lib/python3.8/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/opt/anaconda3/lib/python3.8/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not mis

In [17]:
print(DyConcatMI.shape)

(473, 7, 7)


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
#3.7 FFT-based sum batch's MI dynamic adjacency matrix
print(COVID_features_aftslid.shape)
T=[20,40,100];
Term1=T[0]
Term2=T[1]
Term3=T[2]
#Step1: Calculate Adj in each batch
#(1) Term1
a=[]
from math import ceil#Eg:Ceil(2.2)=3
for i in range(COVID_features_aftslid.shape[2]):
    for j in range(COVID_features_aftslid.shape[2]):
        for k in np.arange(ceil(COVID_features_aftslid.shape[0]/Term1)):
            a.append(Global_MeanMI(COVID_features_aftslid[k*Term1:k*Term1+Term1,:,i],COVID_features_aftslid[k*Term1:k*Term1+Term1,:,j],2))
a=np.array(a).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2],ceil(COVID_features_aftslid.shape[0]/Term1))#a:(7, 7, 23)
a=a.transpose(2,0,1)
print(a.shape)#(24,7,7)

#(2) Term2
b=[]
for i in range(COVID_features_aftslid.shape[2]):
    for j in range(COVID_features_aftslid.shape[2]):
        for k in np.arange(ceil(COVID_features_aftslid.shape[0]/Term2)):
            b.append(Global_MeanMI(COVID_features_aftslid[k*Term2:k*Term2+Term2,:,i],COVID_features_aftslid[k*Term2:k*Term2+Term2,:,j],2))
b=np.array(b).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2],ceil(COVID_features_aftslid.shape[0]/Term2))#a:(7, 7, 23)
print(b.shape)#(7, 7, 12)
b=b.transpose(2,0,1)
print(b.shape)#(12,7,7)

#(3) Term3
c=[]
for i in range(COVID_features_aftslid.shape[2]):
    for j in range(COVID_features_aftslid.shape[2]):
        for k in np.arange(ceil(COVID_features_aftslid.shape[0]/Term3)):
            c.append(Global_MeanMI(COVID_features_aftslid[k*Term3:k*Term3+Term3,:,i],COVID_features_aftslid[k*Term3:k*Term3+Term3,:,j],2))
c=np.array(c).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2],ceil(COVID_features_aftslid.shape[0]/Term3))#a:(7, 7, 23)
print(c.shape)#(7, 7, 5)
c=c.transpose(2,0,1)
print(c.shape)#(5,7,7)
#Step2:Assign it to particular position:
#Each timestamp from the same batch share the same Adj.
#MeanMI
data=[]
for j in range(a.shape[0]):
    for i in range(Term1):
        data.append(a[j,:,:])
data=np.array(data)
test=data
DyConcatMI1=test[:DyConcatMI.shape[0],:,:]
print(DyConcatMI1.shape)#(473,7,7)

data=[]
for j in range(b.shape[0]):
    for i in range(Term2):
        data.append(b[j,:,:])
data=np.array(data)
test=data
DyConcatMI2=test[:DyConcatMI.shape[0],:,:]
print(DyConcatMI2.shape)#(473,7,7)

data=[]
for j in range(c.shape[0]):
    for i in range(Term3):
        data.append(c[j,:,:])
data=np.array(data)
test=data
DyConcatMI3=test[:DyConcatMI.shape[0],:,:]
print(DyConcatMI3.shape)#(473,7,7)

# Step3: Add all of the adjacency matrices
DyConcatFFTMI=[]
DyConcatFFTMI=DyConcatMI1+DyConcatMI2+DyConcatMI3
print(DyConcatFFTMI.shape)#(473,7,7)

# Step4: Normalize adjacency matrices
from sklearn.preprocessing import MinMaxScaler
for i in range(DyConcatFFTMI.shape[0]):
    scaler = MinMaxScaler(feature_range=(0, 1))
    DyConcatFFTMI[i,:,:] = scaler.fit_transform(DyConcatFFTMI[i,:,:])
print(DyConcatFFTMI)

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(473, 3, 7)
(24, 7, 7)
(7, 7, 12)
(12, 7, 7)
(7, 7, 5)
(5, 7, 7)
(473, 7, 7)
(473, 7, 7)
(473, 7, 7)
(473, 7, 7)
[[[1.00000000e+00 1.51488669e-02 2.35163097e-01 ... 1.19849202e-01
   1.14191984e-01 2.55010071e-01]
  [9.30603854e-02 1.00000000e+00 4.69286219e-02 ... 2.72775644e-01
   3.56092256e-01 1.40144743e-01]
  [3.41511336e-01 1.44275137e-01 1.00000000e+00 ... 1.74739719e-01
   3.03800864e-01 3.42237584e-01]
  ...
  [2.25738593e-01 3.89207377e-01 1.52843120e-01 ... 1.00000000e+00
   2.76002756e-01 2.67835286e-01]
  [0.00000000e+00 1.45129275e-01 0.00000000e+00 ... 0.00000000e+00
   1.00000000e+00 0.00000000e+00]
  [2.21332379e-01 0.00000000e+00 2.17633538e-01 ... 1.32859768e-01
   0.00000000e+00 1.00000000e+00]]

 [[1.00000000e+00 1.51488669e-02 2.35163097e-01 ... 1.19849202e-01
   1.14191984e-01 2.55010071e-01]
  [9.30603854e-02 1.00000000e+00 4.69286219e-02 ... 2.72775644e-01
   3.56092256e-01 1.40144743e-01]
  [3.41511336e-01 1.44275137e-01 1.00000000e+00 ... 1.74739719e-01
   3

In [19]:
#3.8 FFT-based MI-based dynamic adjacency matrix
print(COVID_features_aftslid.shape)
T=[20,40,100];
Term1=T[0]
Term2=T[1]
Term3=T[2]

#Step1: Calculate Adj in each batch

#(1) Term1
a=[]
from math import ceil#Eg:Ceil(2.2)=3
for i in range(COVID_features_aftslid.shape[2]):
    for j in range(COVID_features_aftslid.shape[2]):
        for k in np.arange(ceil(COVID_features_aftslid.shape[0]/Term1)):
            a.append(Global_MeanMI(COVID_features_aftslid[k*Term1:k*Term1+Term1,:,i],COVID_features_aftslid[k*Term1:k*Term1+Term1,:,j],2))
a=np.array(a).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2],ceil(COVID_features_aftslid.shape[0]/Term1))#a:(7, 7, 23)
a=a.transpose(2,0,1)
print(a.shape)#(24,7,7)

#(2) Term2
b=[]
for i in range(COVID_features_aftslid.shape[2]):
    for j in range(COVID_features_aftslid.shape[2]):
        for k in np.arange(ceil(COVID_features_aftslid.shape[0]/Term2)):
            b.append(Global_MeanMI(COVID_features_aftslid[k*Term2:k*Term2+Term2,:,i],COVID_features_aftslid[k*Term2:k*Term2+Term2,:,j],2))
b=np.array(b).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2],ceil(COVID_features_aftslid.shape[0]/Term2))#a:(7, 7, 23)
print(b.shape)#(7, 7, 12)
b=b.transpose(2,0,1)
print(b.shape)#(12,7,7)

#(3) Term3
c=[]
for i in range(COVID_features_aftslid.shape[2]):
    for j in range(COVID_features_aftslid.shape[2]):
        for k in np.arange(ceil(COVID_features_aftslid.shape[0]/Term3)):
            c.append(Global_MeanMI(COVID_features_aftslid[k*Term3:k*Term3+Term3,:,i],COVID_features_aftslid[k*Term3:k*Term3+Term3,:,j],2))
c=np.array(c).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2],ceil(COVID_features_aftslid.shape[0]/Term3))#a:(7, 7, 23)
print(c.shape)#(7, 7, 5)
c=c.transpose(2,0,1)
print(c.shape)#(5,7,7)
#Step2: Calculate MI's MI for periodic term 1
#Term1
temp=[]
for j in range(a.shape[0]):
    for i in range(a.shape[0]):
        temp.append(Global_MeanMI(a[j,:,:],a[i,:,:],2))#The first point MI's MI 
temp=np.array(temp).reshape(a.shape[0],a.shape[0])#24*24

#Step3: Weighted sum for the same periodic term
index_all=np.arange(temp.shape[0]*temp.shape[0]).reshape(a.shape[0],a.shape[0])#Initialization of index_all,shape=24*24
for i in range(temp.shape[0]):
    arrIndex=temp[i,:].argsort()
    temp[i,:]=temp[i,:][arrIndex[::-1]]
    index_all[i,:]=arrIndex
print(temp.shape)#24*24
print(index_all.shape)#24*24

all_term=5#Get the first 5 decomposition
a_all=np.zeros((a.shape[0],COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2]))
for i in range(a.shape[0]):
    for u in range(all_term):
        a_all[i,:,:]=a_all[i,:,:]+a[i,:,:]+index_all[i,u]*temp[i,u]
print(a_all.shape)#(24,7,7)

#Step2: Calculate MI's MI for periodic term 2
#Term2
temp=[]
for j in range(b.shape[0]):
    for i in range(b.shape[0]):
        temp.append(Global_MeanMI(b[j,:,:],b[i,:,:],2))#The first point MI's MI 
temp=np.array(temp).reshape(b.shape[0],b.shape[0])#12*12

#Step3: Weighted sum for the same periodic term
index_all=np.arange(temp.shape[0]*temp.shape[0]).reshape(b.shape[0],b.shape[0])#Initialization of index_all,shape=24*24
for i in range(temp.shape[0]):
    arrIndex=temp[i,:].argsort()
    temp[i,:]=temp[i,:][arrIndex[::-1]]
    index_all[i,:]=arrIndex
print(temp.shape)#12*12
print(index_all.shape)#12*12

all_term=5#Get the first 5 decomposition
b_all=np.zeros((b.shape[0],COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2]))
for i in range(b.shape[0]):
    for u in range(all_term):
        b_all[i,:,:]=b_all[i,:,:]+b[i,:,:]+index_all[i,u]*temp[i,u]
print(b_all.shape)#(12,7,7)

#Step2: Calculate MI's MI for periodic term 3
#Term3
temp=[]
for j in range(c.shape[0]):
    for i in range(c.shape[0]):
        temp.append(Global_MeanMI(c[j,:,:],c[i,:,:],2))#The first point MI's MI 
temp=np.array(temp).reshape(c.shape[0],c.shape[0])#12*12

#Step3: Weighted sum for the same periodic term
index_all=np.arange(temp.shape[0]*temp.shape[0]).reshape(c.shape[0],c.shape[0])#Initialization of index_all,shape=24*24
for i in range(temp.shape[0]):
    arrIndex=temp[i,:].argsort()
    temp[i,:]=temp[i,:][arrIndex[::-1]]
    index_all[i,:]=arrIndex
print(temp.shape)#5*5
print(index_all.shape)#5*5

all_term=5#Get the first 5 decomposition
c_all=np.zeros((c.shape[0],COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2]))
for i in range(c.shape[0]):
    for u in range(all_term):
        c_all[i,:,:]=c_all[i,:,:]+c[i,:,:]+index_all[i,u]*temp[i,u]
print(c_all.shape)#(5,7,7)

#Step4:Assign it to particular position:
#Each timestamp from the same batch share the same Adj.
#MeanMI
data=[]
for j in range(a_all.shape[0]):
    for i in range(Term1):
        data.append(a_all[j,:,:])
data=np.array(data)
test=data
DyConcatMI1=test[:DyConcatMI.shape[0],:,:]
print(DyConcatMI1.shape)#(473,7,7)

data=[]
for j in range(b_all.shape[0]):
    for i in range(Term2):
        data.append(b_all[j,:,:])
data=np.array(data)
test=data
DyConcatMI2=test[:DyConcatMI.shape[0],:,:]
print(DyConcatMI2.shape)#(473,7,7)

data=[]
for j in range(c_all.shape[0]):
    for i in range(Term3):
        data.append(c_all[j,:,:])
data=np.array(data)
test=data
DyConcatMI3=test[:DyConcatMI.shape[0],:,:]
print(DyConcatMI3.shape)#(473,7,7)

# Step5: Add all of the adjacency matrices
DyConcatFFTeachMI=DyConcatMI1+DyConcatMI2+DyConcatMI3
print(DyConcatFFTeachMI.shape)#(473,7,7)

# Step6: Normalize adjacency matrices
from sklearn.preprocessing import MinMaxScaler
for i in range(DyConcatFFTeachMI.shape[0]):
    scaler = MinMaxScaler(feature_range=(0, 1))
    DyConcatFFTeachMI[i,:,:] = scaler.fit_transform(DyConcatFFTeachMI[i,:,:])
print(DyConcatFFTeachMI)


(473, 3, 7)


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(24, 7, 7)
(7, 7, 12)
(12, 7, 7)
(7, 7, 5)
(5, 7, 7)
(24, 24)
(24, 24)
(24, 7, 7)
(12, 12)
(12, 12)
(12, 7, 7)
(5, 5)
(5, 5)
(5, 7, 7)
(473, 7, 7)
(473, 7, 7)
(473, 7, 7)
(473, 7, 7)
[[[1.00000000e+00 1.51488669e-02 2.35163097e-01 ... 1.19849202e-01
   1.14191984e-01 2.55010071e-01]
  [9.30603854e-02 1.00000000e+00 4.69286219e-02 ... 2.72775644e-01
   3.56092256e-01 1.40144743e-01]
  [3.41511336e-01 1.44275137e-01 1.00000000e+00 ... 1.74739719e-01
   3.03800864e-01 3.42237584e-01]
  ...
  [2.25738593e-01 3.89207377e-01 1.52843120e-01 ... 1.00000000e+00
   2.76002756e-01 2.67835286e-01]
  [0.00000000e+00 1.45129275e-01 0.00000000e+00 ... 0.00000000e+00
   1.00000000e+00 0.00000000e+00]
  [2.21332379e-01 0.00000000e+00 2.17633538e-01 ... 1.32859768e-01
   0.00000000e+00 1.00000000e+00]]

 [[1.00000000e+00 1.51488669e-02 2.35163097e-01 ... 1.19849202e-01
   1.14191984e-01 2.55010071e-01]
  [9.30603854e-02 1.00000000e+00 4.69286219e-02 ... 2.72775644e-01
   3.56092256e-01 1.40144743e-01]
 

# Begin with DAMR

In [20]:
##Begin with DAMR
print(COVID_features_aftslid.shape)#473*3*7
print(COVID_X1_all_aftslid.shape)#473*7*30
print(COVID_Y1_all.shape)#(473, 7, 1)
print(Adj_dist.shape)#7*7
# Set Hyperpaprameters
tf.keras.backend.clear_session()#clear all
tf.autograph.set_verbosity(0)#reset to zero
learning_rate = 0.0001
batch_size =24
epochs = 100
seed = 42
verbose = 1

(473, 3, 7)
(473, 7, 30)
(473, 7, 1)
(7, 7)


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [21]:
## Define Model---GAT+GCN+DIFFCONV
#inputs_ret = Input(shape=(COVID_X1_all_aftslid.shape[1], COVID_X1_all_aftslid.shape[2], ))#7*30=node*lags(Time to look back)
inputs_feat = Input(shape=(COVID_features_aftslid.shape[2], COVID_features_aftslid.shape[1], ))#7*6=Node*feature
inputs_adj = Input(shape=(Adj_dist.shape[1],Adj_dist.shape[1], ))#7*7=node*node
#print(inputs_ret.shape)
print(inputs_feat.shape)
print(inputs_adj.shape)

(None, 7, 3)
(None, 7, 7)


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [22]:
#GATinput:1.input nodes*lags 2.adj:nodes*nodes
#GAToutput:1.channels*att_heads2.Att_weights=nodes*heads*nodes
#GAT_output_P, Att_weights_P =  GATConv(16, attn_heads=4, concat_heads=True, dropout_rate=0.3, return_attn_coef=True, 
#                                   activation='relu', use_bias=False)([inputs_ret, inputs_adj])

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [23]:
#Att_weights_P = tf.math.reduce_mean(Att_weights_P, axis=2)#reduce heads,it will be nodes*nodes
#print(Att_weights_P)

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [24]:
GCN_output_P =  GCNConv(16, dropout_rate=0.3, 
                        activation='relu', use_bias=True)([inputs_feat, inputs_adj])

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [25]:
#diff_out = DiffusionConv(16, K=6, activation='tanh')([inputs_feat, Att_weights_P])#diff_out=nodes*channels

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [26]:
#con_out_P_2 = Concatenate(axis=-1)([GAT_output_P, GCN_output_P,diff_out])
dense_P = Dense(8, activation='relu')(GCN_output_P)

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [27]:
dense_out = Dropout(0.2)(dense_P)
outputs = Dense(1)(dense_out)



/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [28]:
model = Model(inputs=[inputs_feat, inputs_adj], outputs=outputs)
model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse', metrics=["mae", "mape"])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 7, 3)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 7, 7)]       0                                            
__________________________________________________________________________________________________
gcn_conv (GCNConv)              (None, 7, 16)        64          input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
dense (Dense)                   (None, 7, 8)         136         gcn_conv[0][0]               

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [29]:
#Get 473*7*7 for static adjacency matrix
#3.1 Adj_dist
Adj_dist=adj_dist(COVID_lat,COVID_long)#7*7
Adj_dist=pd.DataFrame(Adj_dist)
Adj_all = np.arange(473*7*7).reshape(473,7,7)#(473,7,7)
Adj_all[0,:,:]=pd.DataFrame(Adj_all[0,:,:])
Adj_all[0,:,:]=Adj_dist
data=[]
for i in range(Adj_all.shape[0]):
    data.append(Adj_dist)
data=np.array(data)
Adj_dist=data
#3.2 MeanMI
Adj_MeanMI=MeanMI
Adj_all = np.arange(473*7*7).reshape(473,7,7)#(473,7,7)
Adj_all[0,:,:]=pd.DataFrame(Adj_all[0,:,:])
Adj_all[0,:,:]=MeanMI
data=[]
for i in range(Adj_all.shape[0]):
    data.append(Adj_MeanMI)
data=np.array(data)
Adj_MeanMI=data
#3.3 ConcatMI
Adj_ConcatMI=ConcatMI
Adj_all = np.arange(473*7*7).reshape(473,7,7)#(473,7,7)
Adj_all[0,:,:]=pd.DataFrame(Adj_all[0,:,:])
Adj_all[0,:,:]=Adj_ConcatMI
data=[]
for i in range(Adj_all.shape[0]):
    data.append(Adj_ConcatMI)
data=np.array(data)
Adj_ConcatMI=data
#3.4 FeatureMI
Adj_FeatureMI=FeatureMI
Adj_all = np.arange(473*7*7).reshape(473,7,7)#(473,7,7)
Adj_all[0,:,:]=pd.DataFrame(Adj_all[0,:,:])
Adj_all[0,:,:]=Adj_FeatureMI
data=[]
for i in range(Adj_all.shape[0]):
    data.append(Adj_FeatureMI)
data=np.array(data)
Adj_FeatureMI=data
# #3.9 ->MeanMI approach(3.1+3.2+3.5+3.8)
# coe1=0.1
# coe2=0.8
# coe3=0.4
# coe4=0.1
# Adj_all_MeanMI=coe1*Adj_dist+coe2*Adj_MeanMI+coe3*DyMeanMI+coe4*DyConcatFFTeachMI
# scaler = MinMaxScaler(feature_range=(-1, 1))
# for i in range(Adj_all_MeanMI.shape[0]):
#     Adj_all_MeanMI[i,:,:]=scaler.fit_transform(Adj_all_MeanMI[i,:,:])
# #3.10 ->()ConcatMI approach
# Adj_all_ConcatMI=coe1*Adj_dist+coe2*Adj_MeanMI+coe3*DyMeanMI+coe4*DyConcatFFTeachMI
# for i in range(Adj_all_ConcatMI.shape[0]):
#     Adj_all_ConcatMI[i,:,:]=scaler.fit_transform(Adj_all_ConcatMI[i,:,:])

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [30]:
scaler = MinMaxScaler(feature_range=(0, 1))

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [31]:
# #DyMeanMI
# #3.2 Mean MI-based dynamic adjacency matrix
# print(COVID_features_aftslid.shape)#(473, 3, 7)
# #Index all of the rows and columns
# MeanMI=[]
# for k in range(COVID_features_aftslid.shape[0]):
#     for i in range(COVID_features_aftslid.shape[2]):
#         for j in range(COVID_features_aftslid.shape[2]):
#             MeanMI.append(Global_MeanMI(COVID_features_aftslid[:k,:,i],
#                                     COVID_features_aftslid[:k,:,j],2))
# DyMeanMI=np.array(MeanMI).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[0])
# DyMeanMI=DyMeanMI.transpose(2, 0, 1)
# print(DyMeanMI.shape)#(473,7,7)

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [32]:

print(COVID_features_aftslid.shape)#473*3*7
COVID_features_aftslid= np.transpose(COVID_features_aftslid,(0,2,1))#473*7*3
print(COVID_X1_all_aftslid.shape)#473*7*30
print(np.array(COVID_Y1_all).shape)#(473, 7, 1)
print(np.array(Adj_MeanMI).shape)#473*7*7
#Normalization
scaler = MinMaxScaler(feature_range=(-1, 1))
scaled1 = scaler.fit_transform(COVID_Y1_all[:,:,0])
COVID_Y1_all[:,:,0]=scaled1


(473, 3, 7)
(473, 7, 30)
(473, 7, 1)
(473, 7, 7)


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [33]:
tf.keras.backend.clear_session()#clear all
tf.autograph.set_verbosity(0)#reset to zero
learning_rate = 0.0001
batch_size =72
epochs = 100
seed = 42
verbose = 1
ChangeAdj=Adj_FeatureMI
x_train=COVID_X1_all_aftslid[:int(len(COVID_X1_all_aftslid)*0.8),:,:]
x_val=COVID_X1_all_aftslid[int(len(COVID_X1_all_aftslid)*0.8):int(len(COVID_X1_all_aftslid)*0.9),:,:]
x_test=COVID_X1_all_aftslid[int(len(COVID_X1_all_aftslid)*0.9):,:,:]
features_train=COVID_features_aftslid[:int(len(COVID_features_aftslid)*0.8),:,:]
features_val=COVID_features_aftslid[int(len(COVID_features_aftslid)*0.8):int(len(COVID_features_aftslid)*0.9),:,:]
features_test=COVID_features_aftslid[int(len(COVID_features_aftslid)*0.9):,:,:]
adj_train=ChangeAdj[:int(len(ChangeAdj)*0.8),:,:]
adj_val=ChangeAdj[int(len(ChangeAdj)*0.8):int(len(ChangeAdj)*0.9),:,:]
adj_test=ChangeAdj[int(len(ChangeAdj)*0.9):,:,:]
y_train=COVID_Y1_all[:int(len(COVID_Y1_all)*0.8),:,:]
y_val=COVID_Y1_all[int(len(COVID_Y1_all)*0.8):int(len(COVID_Y1_all)*0.9),:,:]
y_test=COVID_Y1_all[int(len(COVID_Y1_all)*0.9):,:,:]

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [34]:
print(x_train.shape)
print(features_train.shape)
print(adj_train.shape)
print(y_train.shape)

(378, 7, 30)
(378, 7, 3)
(378, 7, 7)
(378, 7, 1)


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [35]:
tf.config.experimental_run_functions_eagerly(True)
# Set Hyperpaprameters
tf.keras.backend.clear_session()#clear all
tf.autograph.set_verbosity(0)#reset to zero


# hists = model.fit(x= [x_train, features_train, adj_train],
#                   y= y_train, verbose=verbose, epochs=500,
#                   batch_size=batch_size, #validation_split=0.1, 
#                   validation_data=([x_val, features_val, adj_val], y_val),
#                   #test_data=([x_test, features_test, adj_test], y_test),
#   )

##If you want to train faster, try earlystopping.

hists = model.fit(x= [features_train, adj_train],
                  y= y_train, verbose=verbose, epochs=500,
                  batch_size=batch_size, #validation_split=0.1, 
                  validation_data=([features_val, adj_val], y_val),
                  #test_data=([x_test, features_test, adj_test], y_test),
                    callbacks=[EarlyStopping(#monitor="val_mean_absolute_error", 
                      monitor="val_loss", patience=20, restore_best_weights=True)],
  )


Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py:4211: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py:4211: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf

Epoch 1/500
6/6 [==============================] - 0s 41ms/step - loss: 0.8348 - mae: 0.7328 - mape: 160.2452 - val_loss: 0.2404 - val_mae: 0.4536 - val_mape: 81.4538
Epoch 2/500
4/6 [===================>..........] - ETA: 0s - loss: 0.7820 - mae: 0.7002 - mape: 143.6753

/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py:4211: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py:4211: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py:4211: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.ex

6/6 [==============================] - 0s 30ms/step - loss: 0.7931 - mae: 0.7112 - mape: 148.1429 - val_loss: 0.2373 - val_mae: 0.4506 - val_mape: 80.9947
Epoch 3/500
6/6 [==============================] - 0s 29ms/step - loss: 0.8070 - mae: 0.7185 - mape: 148.6803 - val_loss: 0.2343 - val_mae: 0.4475 - val_mape: 80.5311
Epoch 4/500
6/6 [==============================] - 0s 28ms/step - loss: 0.7825 - mae: 0.7058 - mape: 153.5380 - val_loss: 0.2314 - val_mae: 0.4445 - val_mape: 80.0792
Epoch 5/500
6/6 [==============================] - 0s 39ms/step - loss: 0.7488 - mae: 0.6902 - mape: 147.2096 - val_loss: 0.2287 - val_mae: 0.4417 - val_mape: 79.6618
Epoch 6/500
6/6 [==============================] - 0s 34ms/step - loss: 0.7048 - mae: 0.6716 - mape: 153.8606 - val_loss: 0.2260 - val_mae: 0.4388 - val_mape: 79.2275
Epoch 7/500
6/6 [==============================] - 0s 30ms/step - loss: 0.7036 - mae: 0.6685 - mape: 147.7828 - val_loss: 0.2233 - val_mae: 0.4360 - val_mape: 78.8341
Epoch 8/50

Epoch 51/500
6/6 [==============================] - 0s 32ms/step - loss: 0.2524 - mae: 0.3781 - mape: 102.9991 - val_loss: 0.1401 - val_mae: 0.3352 - val_mape: 68.9074
Epoch 52/500
6/6 [==============================] - 0s 33ms/step - loss: 0.2465 - mae: 0.3721 - mape: 113.7759 - val_loss: 0.1389 - val_mae: 0.3333 - val_mape: 68.6936
Epoch 53/500
6/6 [==============================] - 0s 34ms/step - loss: 0.2311 - mae: 0.3617 - mape: 103.3875 - val_loss: 0.1375 - val_mae: 0.3314 - val_mape: 68.4650
Epoch 54/500
6/6 [==============================] - 0s 39ms/step - loss: 0.2447 - mae: 0.3656 - mape: 102.6529 - val_loss: 0.1362 - val_mae: 0.3295 - val_mape: 68.2366
Epoch 55/500
6/6 [==============================] - 0s 32ms/step - loss: 0.2319 - mae: 0.3586 - mape: 104.0672 - val_loss: 0.1349 - val_mae: 0.3275 - val_mape: 67.9915
Epoch 56/500
6/6 [==============================] - 0s 32ms/step - loss: 0.2379 - mae: 0.3580 - mape: 116.3071 - val_loss: 0.1337 - val_mae: 0.3258 - val_mape: 

Epoch 100/500
6/6 [==============================] - 0s 38ms/step - loss: 0.1549 - mae: 0.2781 - mape: 119.7346 - val_loss: 0.0879 - val_mae: 0.2600 - val_mape: 60.2854
Epoch 101/500
6/6 [==============================] - 0s 31ms/step - loss: 0.1518 - mae: 0.2774 - mape: 130.8428 - val_loss: 0.0872 - val_mae: 0.2589 - val_mape: 60.1529
Epoch 102/500
6/6 [==============================] - 0s 28ms/step - loss: 0.1568 - mae: 0.2774 - mape: 107.9945 - val_loss: 0.0864 - val_mae: 0.2578 - val_mape: 60.0254
Epoch 103/500
6/6 [==============================] - 0s 50ms/step - loss: 0.1541 - mae: 0.2767 - mape: 96.1279 - val_loss: 0.0856 - val_mae: 0.2567 - val_mape: 59.8747
Epoch 104/500
6/6 [==============================] - 0s 36ms/step - loss: 0.1504 - mae: 0.2734 - mape: 108.0409 - val_loss: 0.0848 - val_mae: 0.2556 - val_mape: 59.7399
Epoch 105/500
6/6 [==============================] - 0s 32ms/step - loss: 0.1499 - mae: 0.2773 - mape: 122.4479 - val_loss: 0.0841 - val_mae: 0.2546 - val_m

6/6 [==============================] - 0s 27ms/step - loss: 0.1028 - mae: 0.2432 - mape: 125.4050 - val_loss: 0.0527 - val_mae: 0.1970 - val_mape: 50.1229
Epoch 197/500
6/6 [==============================] - 0s 24ms/step - loss: 0.0993 - mae: 0.2398 - mape: 115.2299 - val_loss: 0.0526 - val_mae: 0.1965 - val_mape: 50.0154
Epoch 198/500
6/6 [==============================] - 0s 27ms/step - loss: 0.1041 - mae: 0.2444 - mape: 108.1199 - val_loss: 0.0524 - val_mae: 0.1961 - val_mape: 49.9058
Epoch 199/500
6/6 [==============================] - 0s 34ms/step - loss: 0.1070 - mae: 0.2433 - mape: 125.8021 - val_loss: 0.0522 - val_mae: 0.1957 - val_mape: 49.8078
Epoch 200/500
6/6 [==============================] - 0s 32ms/step - loss: 0.0990 - mae: 0.2386 - mape: 123.5687 - val_loss: 0.0520 - val_mae: 0.1953 - val_mape: 49.7160
Epoch 201/500
6/6 [==============================] - 0s 27ms/step - loss: 0.1055 - mae: 0.2456 - mape: 115.0921 - val_loss: 0.0519 - val_mae: 0.1951 - val_mape: 49.6581


6/6 [==============================] - 0s 29ms/step - loss: 0.0776 - mae: 0.2150 - mape: 110.2403 - val_loss: 0.0347 - val_mae: 0.1310 - val_mape: 40.6257
Epoch 293/500
6/6 [==============================] - 0s 26ms/step - loss: 0.0788 - mae: 0.2189 - mape: 107.6221 - val_loss: 0.0347 - val_mae: 0.1310 - val_mape: 40.6766
Epoch 294/500
6/6 [==============================] - 0s 24ms/step - loss: 0.0778 - mae: 0.2139 - mape: 106.4247 - val_loss: 0.0348 - val_mae: 0.1311 - val_mape: 40.7930
Epoch 295/500
6/6 [==============================] - 0s 25ms/step - loss: 0.0806 - mae: 0.2187 - mape: 111.2065 - val_loss: 0.0349 - val_mae: 0.1313 - val_mape: 40.9145
Epoch 296/500
6/6 [==============================] - 0s 26ms/step - loss: 0.0746 - mae: 0.2100 - mape: 113.8535 - val_loss: 0.0349 - val_mae: 0.1314 - val_mape: 40.9989
Epoch 297/500
6/6 [==============================] - 0s 25ms/step - loss: 0.0759 - mae: 0.2132 - mape: 116.6541 - val_loss: 0.0348 - val_mae: 0.1313 - val_mape: 41.1221


In [36]:
def mae(y_true, y_pred):
    return np.mean(np.abs(y_pred - y_true))

def rmse(y_true, y_pred):
    return np.sqrt(np.mean(np.square(y_pred - y_true)))

def mape(y_true, y_pred, threshold=0.1):
    v = np.clip(np.abs(y_true), threshold, None)
    diff = np.abs((y_true - y_pred) / v)
    return np.mean(diff, axis=-1).mean()
###Write the function of mre
def mre(y_true, y_pred):
    return np.sum(np.abs(y_pred - y_true))/np.sum(np.abs(y_true))
def get_metrics(y, yp):
    return {

        "mae": np.round(mae(y, yp), 4),
        "mape": np.round(mape(y, yp),4),
        "mre": np.round(mre(y, yp),4),
        "rmse": np.round(rmse(y, yp), 4),
        #"MPE": np.round(MPE(y, yp), 4),
        #"R2": np.round(R_squared(y, yp),4)   
    }

# scaled1 = scaler.fit_transform(MeanMI)
#Get the tensor of test set after masking it.
test_predicted = model.predict([features_test, adj_test])
get_metrics(y_test.flatten(), test_predicted.flatten())


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py:4211: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py:4211: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf

{'mae': 0.1761, 'mape': 0.2064, 'mre': 0.2054, 'rmse': 0.2715}

In [37]:
#Get the mask of y_test.
Mask_BZ=pd.read_csv("Data/COVID/10%/BZ_10.csv")
Mask_CL=pd.read_csv("Data/COVID/10%/CL_10.csv")
Mask_ID=pd.read_csv("Data/COVID/10%/ID_10.csv")
Mask_IR=pd.read_csv("Data/COVID/10%/IR_10.csv")
Mask_MX=pd.read_csv("Data/COVID/10%/MX_10.csv")
Mask_PR=pd.read_csv("Data/COVID/10%/PR_10.csv")
Mask_SA=pd.read_csv("Data/COVID/10%/SA_10.csv")

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [38]:
Mask_BZ=np.array(Mask_BZ)
missing_mask=[]
for i in range(Mask_BZ.shape[0]):
    missing_mask.append(np.isnan(np.array(Mask_BZ[i])))
missing_mask=np.array(missing_mask).reshape(Mask_BZ.shape[0],Mask_BZ.shape[1])
missing_mask_BZ_test=missing_mask[-15-y_test.shape[0]:-15,:]#48*3
#Get the first feature's mask of BZ
missing_mask_BZ_test1=missing_mask_BZ_test[:,0]

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [39]:
Mask_CL=np.array(Mask_CL)
missing_mask=[]
for i in range(Mask_CL.shape[0]):
    missing_mask.append(np.isnan(np.array(Mask_CL[i])))
missing_mask=np.array(missing_mask).reshape(Mask_CL.shape[0],Mask_CL.shape[1])
missing_mask_CL_test=missing_mask[-15-y_test.shape[0]:-15,:]#48*3
#Get the first feature's mask of CL
missing_mask_CL_test1=missing_mask_CL_test[:,0]
Mask_ID=np.array(Mask_ID)
missing_mask=[]
for i in range(Mask_ID.shape[0]):
    missing_mask.append(np.isnan(np.array(Mask_ID[i])))
missing_mask=np.array(missing_mask).reshape(Mask_ID.shape[0],Mask_ID.shape[1])
missing_mask_ID_test=missing_mask[-15-y_test.shape[0]:-15,:]#48*3
#Get the first feature's mask of ID
missing_mask_ID_test1=missing_mask_ID_test[:,0]
Mask_IR=np.array(Mask_IR)
missing_mask=[]
for i in range(Mask_IR.shape[0]):
    missing_mask.append(np.isnan(np.array(Mask_IR[i])))
missing_mask=np.array(missing_mask).reshape(Mask_IR.shape[0],Mask_IR.shape[1])
missing_mask_IR_test=missing_mask[-15-y_test.shape[0]:-15,:]#48*3
#Get the first feature's mask of IR
missing_mask_IR_test1=missing_mask_IR_test[:,0]
Mask_MX=np.array(Mask_MX)
missing_mask=[]
for i in range(Mask_MX.shape[0]):
    missing_mask.append(np.isnan(np.array(Mask_MX[i])))
missing_mask=np.array(missing_mask).reshape(Mask_MX.shape[0],Mask_MX.shape[1])
missing_mask_MX_test=missing_mask[-15-y_test.shape[0]:-15,:]#48*3
#Get the first feature's mask of MX
missing_mask_MX_test1=missing_mask_MX_test[:,0]
Mask_PR=np.array(Mask_PR)
missing_mask=[]
for i in range(Mask_PR.shape[0]):
    missing_mask.append(np.isnan(np.array(Mask_PR[i])))
missing_mask=np.array(missing_mask).reshape(Mask_PR.shape[0],Mask_PR.shape[1])
missing_mask_PR_test=missing_mask[-15-y_test.shape[0]:-15,:]#48*3
#Get the first feature's mask of PR
missing_mask_PR_test1=missing_mask_PR_test[:,0]
Mask_SA=np.array(Mask_SA)
missing_mask=[]
for i in range(Mask_SA.shape[0]):
    missing_mask.append(np.isnan(np.array(Mask_SA[i])))
missing_mask=np.array(missing_mask).reshape(Mask_SA.shape[0],Mask_SA.shape[1])
missing_mask_SA_test=missing_mask[-15-y_test.shape[0]:-15,:]#48*3
#Get the first feature's mask of SA
missing_mask_SA_test1=missing_mask_SA_test[:,0]

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [40]:
missing_mask_ALL_test1=np.stack((missing_mask_BZ_test1,missing_mask_CL_test1
            ,missing_mask_ID_test1,missing_mask_IR_test1,
            missing_mask_MX_test1,missing_mask_PR_test1,missing_mask_SA_test1
           ),axis=1)

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [41]:
#Get test set result.
get_metrics(y_test[missing_mask_ALL_test1].flatten(), test_predicted[missing_mask_ALL_test1].flatten())

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'mae': 0.1408, 'mape': 0.1613, 'mre': 0.1646, 'rmse': 0.2307}